# Replicate Fig7D from Rees et al 2016


In [1]:
# from igraph import *
# from bluepy.enums import Cell
from scipy import sparse
import bluepy
from tqdm import tqdm
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import sys
import logging
from graph_analysis.triplets import MotifReader, CA1MotifCalculator

import networkx as nx
from dotmotif import Motif, GrandIsoExecutor
from graph_analysis import triplets

In [2]:
all_motifs = ['-C','-B','-A','A','B','C','D','E','F','G','H','I','J','K','L','M']
save_dir = '../../output/triplets'
nodeset = 'cylinder300'

In [3]:
complex_motif = Motif("""
# One-direction edge
uniedge(n, m) {
    n -> m
    m !> n
}

# One-direction triangle
unitriangle(x, y, z) {
    uniedge(x, y)
    uniedge(y, z)
    uniedge(z, x)
}

unitriangle(A, B, C)
""") # specific case not like motif2 where edges can also be biedge/uniedge

In [4]:
file_path = '/gpfs/bbp.cscs.ch/project/proj142/home/kurban/GNNplayground/notebooks/cylinder300_synaptome.npz'
target_adj = sparse.load_npz(file_path)

In [5]:
from bluepysnap import Circuit

c = Circuit('/gpfs/bbp.cscs.ch/project/proj112/circuits/CA1/20211110-BioM/sonata/circuit_config.json')
nodes = c.nodes['hippocampus_neurons']

In [7]:
nodeset_ids = nodes.ids(nodeset)

In [27]:
check_mtypes = ['SP_PC','SP_PC','SP_PC']

In [30]:
assert len(np.unique(check_mtypes)) == 1

In [9]:
nodeset_subids = np.intersect1d(nodeset_ids,nodes.ids('SP_PC')) # to be changed for each, union operator if multiple mtypes

In [18]:
local_indices_selected = np.where(np.isin(nodeset_ids,nodeset_subids))[0] # indices inside nodeset indices

In [23]:
subset_adj = target_adj[local_indices_selected,:][:,local_indices_selected]

In [37]:
subset_adj[subset_adj>1] = 1  # connectome

In [38]:
target_graph = nx.from_scipy_sparse_array(subset_adj,create_using=nx.DiGraph())

In [40]:
Executor = GrandIsoExecutor(graph=target_graph)

In [42]:
frequencies_exc = {}
motif_reader = triplets.MotifReader()
reader = triplets.DotMotifReader()

for motif_name in tqdm(all_motifs):
    logging.info(f"Running motif calculation for {nodeset} of motif {motif_name} of {check_mtypes}...")
    CM = motif_reader.name_to_matrix(motif_name)
    cur_triplet_motif = reader.matrix_to_dotmotif(CM)
    if cur_triplet_motif == '':
        continue
    else:
        logging.info(f"\n{CM} \n\n dotmotif:\n{cur_triplet_motif}\n\n")
        cur_triplet_motif = Motif(cur_triplet_motif)

    cur_results = Executor.find(cur_triplet_motif)
    frequencies_exc[motif_name] = cur_results

  0%|          | 0/16 [00:00<?, ?it/s]INFO:root:Running motif calculation for cylinder300 of motif -C of ['SP_PC', 'SP_PC', 'SP_PC']...
INFO:root:Running motif calculation for cylinder300 of motif -B of ['SP_PC', 'SP_PC', 'SP_PC']...
INFO:root:
[[0 1 0]
 [0 0 0]
 [0 0 0]] 

 dotmotif:
A -> B


 12%|█▎        | 2/16 [00:04<00:32,  2.33s/it]INFO:root:Running motif calculation for cylinder300 of motif -A of ['SP_PC', 'SP_PC', 'SP_PC']...
INFO:root:
[[0 1 0]
 [1 0 0]
 [0 0 0]] 

 dotmotif:
A -> B
B -> A


 19%|█▉        | 3/16 [00:04<00:19,  1.49s/it]INFO:root:Running motif calculation for cylinder300 of motif A of ['SP_PC', 'SP_PC', 'SP_PC']...
INFO:root:
[[0 0 0]
 [1 0 0]
 [1 0 0]] 

 dotmotif:
B -> A
C -> A


 25%|██▌       | 4/16 [28:52<2:09:07, 645.64s/it]INFO:root:Running motif calculation for cylinder300 of motif B of ['SP_PC', 'SP_PC', 'SP_PC']...
INFO:root:
[[0 0 0]
 [1 0 0]
 [0 1 0]] 

 dotmotif:
B -> A
C -> B


 31%|███▏      | 5/16 [43:24<2:12:47, 724.31s/it]INFO:root:Running m

In [45]:
#SAVE 

target_motifs_exc = {}
for i in all_motifs:
    try:
        cur_occurences = len(frequencies_exc[i])
        target_motifs_exc[i] = cur_occurences
    except:
        print(f'{i} not in frequencies')
        target_motifs_exc[i] = -1

-C not in frequencies


In [48]:
import pickle 

with open(f'{save_dir}/exc_triplets_{nodeset}.pkl','wb') as f:
    pickle.dump(frequencies_exc,f)

In [46]:
target_motifs_exc

{'-C': -1,
 '-B': 555057,
 '-A': 10738,
 'A': 75188930,
 'B': 48277113,
 'C': 72068488,
 'D': 1170657,
 'E': 3339580,
 'F': 1285219,
 'G': 391599,
 'H': 107978,
 'I': 31840,
 'J': 33708,
 'K': 103664,
 'L': 2534,
 'M': 156}

# CA1 PYR:INT

In [ ]:
nodeset_subids = np.intersect1d(nodeset_ids,nodes.ids('INT')) # to be changed for each, union operator if multiple mtypes
local_indices_selected = np.where(np.isin(nodeset_ids,nodeset_subids))[0] # indices inside nodeset indices
subset_adj = target_adj[local_indices_selected,:][:,local_indices_selected]
subset_adj[subset_adj>1] = 1  # connectome
target_graph = nx.from_scipy_sparse_array(subset_adj,create_using=nx.DiGraph())
Executor = GrandIsoExecutor(graph=target_graph)

In [ ]:
frequencies_inh = {}

for motif_name in tqdm(all_motifs):
    logging.info(f"Running motif calculation for {nodeset} of motif {motif_name} of {check_mtypes}...")
    CM = motif_reader.name_to_matrix(motif_name)
    cur_triplet_motif = reader.matrix_to_dotmotif(CM)
    if cur_triplet_motif == '':
        continue
    else:
        logging.info(f"\n{CM} \n\n dotmotif:\n{cur_triplet_motif}\n\n")
        cur_triplet_motif = Motif(cur_triplet_motif)

    cur_results = Executor.find(cur_triplet_motif)
    frequencies_inh[motif_name] = cur_results

In [ ]:
# check_mtypes = ['SP_PC','SP_PC','SP_PC']
# compare_to = ['INT','INT','INT']


# target_motifs_exc = {}
# target_motifs_inh = {}

# for motif_name in tqdm(all_motifs):
#     logging.info(f"Running motif calculation for {target} of motif {motif_name} of {check_mtypes}...")
#     motif_reader = MotifReader()
#     CM = motif_reader.name_to_matrix(motif_name)
#     # logging.info(f"Motif matrix:\n{CM}")
# #     calculator = CA1MotifCalculator(target,check_mtypes, CM)
# #     num_motifs = calculator.count_motifs()  # proj samples not used for local computations
    

# #     logging.info(f"Running motif calculation for {target} of motif {motif_name} of {compare_to}...")

#     calculator2 = CA1MotifCalculator(target,compare_to, CM)
#     num_motifs2 = calculator2.count_motifs()  # proj samples not used for local computations

#     target_motifs_exc[motif_name] = num_motifs
#     target_motifs_inh[motif_name] = num_motifs2

In [ ]:
df1 = pd.DataFrame.from_dict(target_motifs_exc, orient='index',columns=[f'{target}_EXC'])
df2 = pd.DataFrame.from_dict(target_motifs_inh, orient='index',columns=[f'{target}_INH'])

df_ei = pd.concat([df1,df2],axis=1)

In [ ]:
df_ei['ratio'] = df_ei[f'{target}_EXC']/df_ei[f'{target}_INH']

In [ ]:
from bluepysnap import Circuit
c =Circuit('/gpfs/bbp.cscs.ch/project/proj112/circuits/CA1/20211110-BioM/sonata/circuit_config.json')
nodes = c.nodes['hippocampus_neurons']
target_nodes = nodes.get(target)

# count how many morph_class is INT
num_int = target_nodes[target_nodes['morph_class']=='INT'].shape[0]
num_exc = target_nodes[target_nodes['morph_class']=='PYR'].shape[0]
print(f"Number of INT cells: {num_int}")
print(f"Number of EXC cells: {num_exc}")

In [ ]:
def max_possible_triplet(num_cells):
    return num_cells*(num_cells-1)*(num_cells-2)/6

In [ ]:
#normalize exc and inh values with max possible values
df_ei['ratio_norm'] = df_ei['ratio'] * (max_possible_triplet(num_int) / max_possible_triplet(num_exc))

In [ ]:
df_ei.to_csv(f'{save_dir}/{target}_triplet_ratios_EI.csv')

In [ ]:
chosen_df = df_ei.ratio[3:]
chosen_df.plot(figsize=(10,5),title=f'{target} motif ratio')
# plot all xticklabels
plt.xticks(np.arange(len(df_ei)-3), df_ei.index[3:])
# horizontal line at y=1
plt.axhline(y=1, color='r', linestyle='-')
plt.ylabel('Normalized Ratio')
plt.savefig(f'{save_dir}/{target}_triplet_ratios_EI.png',dpi=300)
plt.show()

chosen_df.plot(figsize=(10,5),title=f'{target} motif ratio')
# plot all xticklabels
plt.xticks(np.arange(len(df_ei)-3), df_ei.index[3:])
# horizontal line at y=1
plt.yscale('log')
plt.axhline(y=1, color='r', linestyle='-')
plt.ylabel('Normalized Ratio')
plt.savefig(f'{save_dir}/{target}_triplet_ratios_EI_log.png',dpi=300)
plt.show()


In [ ]:
chosen_df = df_ei.ratio_norm[3:]
chosen_df.plot(figsize=(10,5),title=f'{target} motif ratio')
# plot all xticklabels
plt.xticks(np.arange(len(df_ei)-3), df_ei.index[3:])
# horizontal line at y=1
plt.axhline(y=1, color='r', linestyle='-')
plt.ylabel('Normalized Ratio')
plt.show()
plt.savefig(f'{save_dir}/{target}_motif_ratio_EI.png')

chosen_df.plot(figsize=(10,5),title=f'{target} motif ratio')
# plot all xticklabels
plt.xticks(np.arange(len(df_ei)-3), df_ei.index[3:])
# horizontal line at y=1
plt.yscale('log')
plt.axhline(y=1, color='r', linestyle='-')
plt.ylabel('Normalized Ratio')
plt.show()


# CA1 Perisomatic INT: Distal INT

In [ ]:
check_mtypes = ['PeriSomatic_INH','PeriSomatic_INH','PeriSomatic_INH']
compare_to = ['Distal_INH','Distal_INH','Distal_INH']

target_motifs_perisomatic = {}
target_motifs_distal = {}

for motif_name in tqdm(all_motifs):
    logging.info(f"Running motif calculation for {target} of motif {motif_name} of {check_mtypes}...")
    motif_reader = MotifReader()
    CM = motif_reader.name_to_matrix(motif_name)
    # logging.info(f"Motif matrix:\n{CM}")
    calculator = CA1MotifCalculator(target,check_mtypes, CM)
    num_motifs = calculator.count_motifs()  # proj samples not used for local computations

    logging.info(f"Running motif calculation for {target} of motif {motif_name} of {compare_to}...")

    calculator2 = CA1MotifCalculator(target,compare_to, CM)
    num_motifs2 = calculator2.count_motifs()  # proj samples not used for local computations

    target_motifs_perisomatic[motif_name] = num_motifs
    target_motifs_distal[motif_name] = num_motifs2

In [ ]:
df1 = pd.DataFrame.from_dict(target_motifs_perisomatic, orient='index',columns=[f'{target}_perisomatic'])
df2 = pd.DataFrame.from_dict(target_motifs_distal, orient='index',columns=[f'{target}_distal'])

df_peridistal = pd.concat([df1,df2],axis=1)
df_peridistal['ratio'] = df_peridistal[f'{target}_perisomatic']/df_peridistal[f'{target}_distal']
df_peridistal.to_csv(f'{save_dir}/{target}_triplet_ratios_peridistal.csv')

In [ ]:
chosen_df = df_peridistal.ratio[3:]
chosen_df.plot(figsize=(10,5),title=f'{target} motif ratio')
# plot all xticklabels
plt.xticks(np.arange(len(df_peridistal)-3), df_peridistal.index[3:])
# horizontal line at y=1
plt.axhline(y=1, color='r', linestyle='-')
plt.ylabel('Ratio')
plt.savefig(f'{save_dir}/{target}_triplet_ratios_peridistal.png',dpi=300)
plt.show()

chosen_df.plot(figsize=(10,5),title=f'{target} motif ratio')
# plot all xticklabels
plt.xticks(np.arange(len(df_peridistal)-3), df_peridistal.index[3:])
# horizontal line at y=1
plt.yscale('log')
plt.axhline(y=1, color='r', linestyle='-')
plt.ylabel('Ratio')
plt.savefig(f'{save_dir}/{target}_triplet_ratios_peridistal_log.png',dpi=300)
plt.show()


# Combined figure 7D

In [ ]:
df_ei = pd.read_csv(f'{save_dir}/{target}_triplet_ratios_EI.csv',index_col=0)
df_peridistal = pd.read_csv(f'{save_dir}/{target}_triplet_ratios_peridistal.csv',index_col=0)

In [ ]:
chosen_df1 = df_ei.ratio[3:]
# chosen_df1.plot(figsize=(10,5),title=f'{target} motif ratio')
# add black triangles and blue lines connecting for every point in the plot and lines
chosen_df1.plot(style='^-',ms=10,figsize=(10,5))

chosen_df2= df_peridistal.ratio[3:]
chosen_df2.plot(style='o-r',ms=10,figsize=(10,5))


# plot all xticklabels
plt.xticks(np.arange(len(df_peridistal)-3), df_peridistal.index[3:])
# horizontal line at y=1
plt.axhline(y=1, color='k', linestyle='-')
plt.ylabel('Ratio')
plt.legend(['CA1 Pyr:Int','CA1 PST:Other'])
plt.savefig(f'{save_dir}/{target}_Rees2016_Fig7D.png',dpi=300)
plt.show()
